<a href="https://colab.research.google.com/github/CamilleLHermine/Fine_tuning_and_patching_raw_vibration/blob/main/Vibration_data_based_finetuning_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os
import requests

import zipfile
import glob
from scipy.io import loadmat # To read .mat files

import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from urllib3.exceptions import InsecureRequestWarning

import accelerate
import torch
import transformers
from datasets import load_dataset, Dataset, load_from_disk, concatenate_datasets, Features, Value, Sequence, ClassLabel
import datasets

from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback

from google.colab import files
from google.colab import drive

from torch import nn
from torch.nn import functional as F
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torch.nn.modules.loss import CrossEntropyLoss

from IPython.core.debugger import set_trace
from pathlib import Path

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
zip_path = '/content/drive/MyDrive/CWRUDataset.zip'
extract_dir = 'extracted_mat_files/'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [5]:
!ls '/content/extracted_mat_files/CWRU Dataset/Data/12k_FE'
!ls '/content/extracted_mat_files/CWRU Dataset/Data/48k_DE'
!ls '/content/extracted_mat_files/CWRU Dataset/Data/12k_DE'
!ls '/content/extracted_mat_files/CWRU Dataset/Data/Normal'

B007_0.mat  B021_1.mat	 IR014_2.mat	 OR007@12_3.mat  OR014@3_0.mat
B007_1.mat  B021_2.mat	 IR014_3.mat	 OR007@3_0.mat	 OR014@3_1.mat
B007_2.mat  B021_3.mat	 IR021_0.mat	 OR007@3_1.mat	 OR014@3_2.mat
B007_3.mat  IR007_0.mat  IR021_1.mat	 OR007@3_2.mat	 OR014@3_3.mat
B014_0.mat  IR007_1.mat  IR021_2.mat	 OR007@3_3.mat	 OR014@6_0.mat
B014_1.mat  IR007_2.mat  IR021_3.mat	 OR007@6_0.mat	 OR021@3_1.mat
B014_2.mat  IR007_3.mat  OR007@12_0.mat  OR007@6_1.mat	 OR021@3_2.mat
B014_3.mat  IR014_0.mat  OR007@12_1.mat  OR007@6_2.mat	 OR021@3_3.mat
B021_0.mat  IR014_1.mat  OR007@12_2.mat  OR007@6_3.mat	 OR021@6_0.mat
B007_0.mat  B014_0.mat	B021_0.mat  IR007_0.mat  IR014_0.mat  IR021_0.mat
B007_1.mat  B014_1.mat	B021_1.mat  IR007_1.mat  IR014_1.mat  IR021_1.mat
B007_2.mat  B014_2.mat	B021_2.mat  IR007_2.mat  IR014_2.mat  IR021_2.mat
B007_3.mat  B014_3.mat	B021_3.mat  IR007_3.mat  IR014_3.mat  IR021_3.mat
B007_0.mat  B028_0.mat	 IR021_0.mat	 OR007@3_0.mat	OR021@12_0.mat
B007_1.mat  B028_1.mat	 IR021_1.

In [6]:
mat_data = loadmat('/content/extracted_mat_files/CWRU Dataset/Data/48k_DE/B007_0.mat')
mat_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 15:29:05 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X122_DE_time': array([[-0.111192  ],
        [-0.08302892],
        [-0.04234892],
        ...,
        [ 0.02586831],
        [-0.02837169],
        [-0.06759138]]),
 'X122_FE_time': array([[-0.09512545],
        [-0.07211455],
        [-0.02609273],
        ...,
        [ 0.15409091],
        [ 0.07642909],
        [-0.01006727]]),
 'X122RPM': array([[1796]], dtype=uint16)}

In [7]:
import re

def extract_data_from_mat(file_path, key=None):
    mat_data = loadmat(file_path)
    # Remove metadata keys that start with "__"
    mat_keys = [k for k in mat_data.keys() if not k.startswith("__")]

    if key is None:
        # Pick first array-like key
        key = mat_keys[0]

    data = mat_data[key]

    # Convert to torch tensor
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    else:
        data = data[:, 0].reshape(-1, 1)
        data = data.reshape(-1, 1)


    if os.path.basename(file_path).startswith('B'):
      label=('Ball')
    elif os.path.basename(file_path).startswith('IR'):
      label=('Inner race')
    elif os.path.basename(file_path).startswith('OR'):
      label=('Outer race')
    elif os.path.basename(file_path).startswith('Normal'):
      label=('Normal')



    return data, label


In [8]:
def load_folder(folder_path, key=None):
    all_data = []
    labels=[]

    for filename in os.listdir(folder_path):
        if filename.endswith(".mat"):
            file_path = os.path.join(folder_path, filename)
            data,label = extract_data_from_mat(file_path, key=key)
            all_data.append(data)
            labels.append(label)

    return all_data,labels


In [9]:
file_paths = ['/content/extracted_mat_files/CWRU Dataset/Data/48k_DE/', '/content/extracted_mat_files/CWRU Dataset/Data/12k_DE/', '/content/extracted_mat_files/CWRU Dataset/Data/12k_FE/','/content/extracted_mat_files/CWRU Dataset/Data/Normal/']
all_data = []
all_labels = []
freq = []
for folder in file_paths:
    data, labels = load_folder(folder)
    if all_data:
      all_data=all_data+data
      all_labels=all_labels+labels

    else:
      all_data=data
      all_labels=labels


In [10]:
max_length=2048
step=512
input_chunk=[]
target_chunk=[]
for i in range(0,len(all_labels)):
  for j in range(0,len(all_data[i])-max_length,step):
    if len(all_data[i][j:j+max_length])>0:
      input_chunk.append(all_data[i][j:j+max_length])
      target_chunk.append(all_labels[i])

In [11]:
print(len(input_chunk[0]))

2048


In [12]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

numeric_labels = encoder.fit_transform(target_chunk)

print(f"Classes (mapping): {encoder.classes_}")

Classes (mapping): ['Ball' 'Inner race' 'Normal' 'Outer race']


In [13]:
# --- 1. Define Variables ---
window_size = 2048
patch_size = 128
stride = 8
num_patches = (window_size - patch_size) // stride + 1

In [14]:

X = np.array(input_chunk).reshape(-1, window_size)
y = np.array(numeric_labels)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [15]:
len(X_train)

36906

In [16]:
def instance_normalize(data):
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    # small epsilon to avoid division by zero
    return (data - mean) / (std + 1e-6)

X_train_norm = instance_normalize(X_train)
X_val_norm = instance_normalize(X_val)
X_test_norm = instance_normalize(X_test)


In [17]:
len(X_train_norm)

36906

In [18]:
def create_patches(data, patch_size, stride):
    num_samples, window_length = data.shape
    num_patches_per_window = (window_length - patch_size) // stride + 1

    patches = np.zeros((num_samples, num_patches_per_window, patch_size))

    for i in range(num_samples):
        for j in range(num_patches_per_window):
            start_index = j * stride
            end_index = start_index + patch_size
            patches[i, j, :] = data[i, start_index:end_index]
    return patches

X_train_patched = create_patches(X_train_norm, patch_size, stride)
X_val_patched = create_patches(X_val_norm, patch_size, stride)
X_test_patched = create_patches(X_test_norm, patch_size, stride)

# Check the final shape: (num_samples, num_patches, patch_size)
print(f"Shape of patched training data: {X_train_patched.shape}")
print(f"Shape of patched validation data: {X_val_patched.shape}")
print(f"Shape of patched testing data: {X_test_patched.shape}")

Shape of patched training data: (36906, 241, 128)
Shape of patched validation data: (4101, 241, 128)
Shape of patched testing data: (4557, 241, 128)


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score

X_train_tensor = torch.tensor(X_train_patched, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val_patched, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_patched, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        """
        Args:
            d_model (int): The dimension of the model's embeddings.
            max_len (int): The maximum possible sequence length.
        """
        super().__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # div of 1 / (10000^(2i/d_model))
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): The input tensor (token embeddings) of shape
                              (batch_size, sequence_length, d_model).
        """
        # Add the positional encoding to the input tensor.

        x = x + self.pe[:x.size(1), :]
        return x


In [21]:
#'token_embeddings' has shape (batch_size, num_patches, d_model)

In [22]:
from transformers import GPT2Model, GPT2Config

d_model = 768 # d_model for GPT-2
num_classes = len(encoder.classes_) # Number of fault types

class VibrationLLM(nn.Module):
    def __init__(self, patch_size, d_model, num_patches, num_classes, config):
        super().__init__()

        # This converts each patch of size 128 into a vector of size d_model (768)
        self.token_embedder = nn.Conv1d(in_channels=1, out_channels=config.d_model, kernel_size=config.patch_size, stride=1)
        self.config = config
        self.pos_encoder = PositionalEncoding(d_model=config.d_model, max_len=config.num_patches + 1)
        self.llm_body = GPT2Model.from_pretrained('gpt2')

        # FREEZE LLM body
        for param in self.llm_body.parameters():
            param.requires_grad = False

        # Classification Head
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, input_ids, **kwargs):
        x=input_ids
        # Input x shape: (batch_size, num_patches, patch_size)
        # Reshape for Conv1D: (batch_size * num_patches, 1, patch_size)
        batch_size, num_patches, patch_size = x.shape
        x_reshaped = x.view(-1, 1, patch_size)

        token_embeddings = self.token_embedder(x_reshaped)
        token_embeddings = token_embeddings.view(batch_size, num_patches, -1)

        final_embeddings = self.pos_encoder(token_embeddings)

        llm_output = self.llm_body(inputs_embeds=final_embeddings)
        hidden_states = llm_output.last_hidden_state

        # We use the hidden state of the first patch for classification
        pooled_output = hidden_states[:, 0]
        logits = self.classifier(pooled_output)

        return logits

In [23]:
from peft import get_peft_model, LoraConfig, TaskType

class ModelConfig:
    """A simple config class that allows both attribute and dictionary-style access."""
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def get(self, key, default=None):
        return getattr(self, key, default)

model_config = ModelConfig(
    patch_size= patch_size,
    d_model= d_model,
    num_patches= num_patches,
    num_classes= num_classes,
    use_return_dict=True,
    tie_word_embeddings= False
)

model = VibrationLLM(
    patch_size=patch_size,
    d_model=d_model,
    num_patches=num_patches,
    num_classes=num_classes,
    config=model_config
)
print(model)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

VibrationLLM(
  (token_embedder): Conv1d(1, 768, kernel_size=(128,), stride=(1,))
  (pos_encoder): PositionalEncoding()
  (llm_body): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): Linear(i

In [24]:
# We need to tell PEFT which parts of the model to adapt.
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    # Specify the modules to unfreeze and train fully (in addition to LoRA modules)
    modules_to_save=['classification',"ln_f", "ln_1", 'ln_2'], # Common names for LayerNorm in transformers
    task_type=TaskType.SEQ_CLS
)
peft_model = get_peft_model(model, lora_config)

peft_model.print_trainable_parameters() #check LoRA is active


trainable params: 852,484 || all params: 125,394,440 || trainable%: 0.6798


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [25]:
checkpoint_dir = "my_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
num_checkpoints = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device) # Use the peft_model from now on


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): VibrationLLM(
      (token_embedder): Conv1d(1, 768, kernel_size=(128,), stride=(1,))
      (pos_encoder): PositionalEncoding()
      (llm_body): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): ModulesToSaveWrapper(
              (original_module): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (modules_to_save): ModuleDict(
                (default): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              )
            )
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (

In [26]:
optimizer = AdamW(peft_model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()
epochs = 5

checkpoint_dir = "/content/drive/MyDrive/checkpoint_dir/"

os.makedirs(checkpoint_dir, exist_ok=True)

optimizer = AdamW(peft_model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()

start_epoch = 0
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_')]

if checkpoint_files:
    latest_checkpoint_path = max(
        [os.path.join(checkpoint_dir, f) for f in checkpoint_files],
        key=os.path.getctime # Use file creation time to find the most recent
    )

    print(f"Resuming training from checkpoint: {latest_checkpoint_path}")
    checkpoint = torch.load(latest_checkpoint_path)

    peft_model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    start_epoch = checkpoint['epoch'] + 1
    last_loss = checkpoint.get('loss', 0.0)
    print(f"Loaded model from epoch {checkpoint['epoch']} with loss {last_loss:.4f}")
else:
    print("No checkpoint found. Starting training from scratch.")

for epoch in range(start_epoch,epochs):
    peft_model.train()
    total_train_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        logits = peft_model(data)
        loss = criterion(logits, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {total_train_loss / len(train_loader):.4f}")

    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_{epoch % num_checkpoints}.pt")

    torch.save({
        'epoch': epoch,
        'model_state_dict': peft_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_train_loss / len(train_loader),
    }, checkpoint_path)

    print(f"Checkpoint saved to {checkpoint_path}")




    peft_model.eval()
    total_val_loss = 0
    correct_predictions = 0
    total_samples = 0

    best_val_loss = float('inf')

    with torch.no_grad(): # Disable gradient calculation to save memory and speed up
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)

            logits = peft_model(data)
            loss = criterion(logits, targets)
            total_val_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_samples += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = (correct_predictions / total_samples) * 100
    print(f"Epoch {epoch+1}/{epochs} -> Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(checkpoint_dir, "best_model.pt")
        torch.save(peft_model.state_dict(), best_model_path)
        print(f"New best model saved to {best_model_path} with validation loss: {best_val_loss:.4f}")

Resuming training from checkpoint: /content/drive/MyDrive/checkpoint_dir/checkpoint_4.pt
Loaded model from epoch 4 with loss 0.8603


In [30]:
peft_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
  for data, targets in test_loader:
      data, targets = data.to(device), targets.to(device)
      logits = peft_model(data)
      preds = torch.argmax(logits, dim=1)
      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(targets.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}\n")

Test Accuracy: 0.6182

